In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.special import softmax as scipy_softmax

In [2]:
df = pd.read_csv('../datasets/processed_dataset.csv')
df

,Course Code,Course Title,Student Number,Department Code,Course Level,Letter Grade,Status,GPA,Standing,Completed Credits,Completed ECTS,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Semester,Theoritical,Practical,Course Credit,ECTS,Course Year
0,UNI 111,Critical Reading & Writing in Turkish I,240,SOC,Undergraduate,F,Unsuccessful,2.62,Freshman,18,30,2.616667,2.113636,2.703226,2011 - Fall,3,0,3,5,1
1,UNI 107,World Civilizations& Global Encounters I,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,30,3.683333,2.986364,2.703226,2011 - Fall,3,0,3,5,1
2,UNI 105,Understanding Society and Culture I,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,30,3.683333,3.211538,2.703226,2011 - Fall,3,0,3,5,1
3,UNI 203,Understanding Science and Technology,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,30,3.683333,3.176000,2.703226,2011 - Fall,3,0,3,5,2
4,UNI 105,Understanding Society and Culture I,240,SOC,Undergraduate,A,Successful,2.62,Freshman,18,30,2.616667,3.211538,2.703226,2011 - Fall,3,0,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48736,MGT 531,Stratejik Yönetim,1102,MBA/NT,Graduate,A-,Successful,3.80,Freshman,24,40,3.800000,3.391758,3.049526,2014 - Spring,3,0,3,5,5
48737,MGT 585,Operasyon Yönetimi,1102,MBA/NT,Graduate,A,Successful,3.80,Freshman,24,40,3.800000,3.440476,3.049526,2014 - Spring,3,0,3,5,5
48738,MGT 552,İnsan Kaynakları Yönetimi,1102,MBA/NT,Graduate,A,Successful,3.80,Freshman,24,40,3.800000,3.393077,3.049526,2014 - Spring,3,0,3,5,5
48739,MGT 574,Makroekonomik Göster. ve Pol. Anal.,1984,MBA/NT,Graduate,A-,Successful,3.20,Sophomore,24,40,3.200000,3.570769,3.049526,2014 - Spring,3,0,3,5,5


In [3]:
df.drop([df.columns[0], df.columns[1], df.columns[2]], inplace=True, axis=1)   # dropping course details

In [4]:
df = pd.concat([df, pd.get_dummies(df['Course Year'], prefix='Course Year'), pd.get_dummies(df['Department Code'], prefix='Department Code'), pd.get_dummies(df['Course Level'], prefix='Course Level'), pd.get_dummies(df['Standing'], prefix='Standing'), pd.get_dummies(df['Status'], prefix='Status')], axis=1)
df.drop(['Course Year', 'Department Code', 'Course Level', 'Status', 'Standing'], axis=1, inplace=True)

In [5]:
columns = df.columns

In [6]:
le = LabelEncoder()
le.fit(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F'])

LabelEncoder()

In [7]:
def standardize(X_train, X_test):
    X_train_cols = X_train.columns
    X_test_cols = X_test.columns
    sc = StandardScaler()
    fitted_sc = sc.fit(X_train)
    X_train_std = pd.DataFrame(fitted_sc.transform(X_train), columns=X_train_cols)
    X_test_std = pd.DataFrame(fitted_sc.transform(X_test), columns=X_test_cols)
    return X_train_std, X_test_std

In [8]:
def get_train_data(df, train_sem, columns):
    dataFrame = pd.DataFrame(columns=columns)
    for sem in train_sem:
        dataFrame = pd.concat([dataFrame, df[df.iloc[:, 7] == sem]], ignore_index=True)
    
    X_train = dataFrame.drop('Semester', axis=1)
    y_train = le.transform(X_train.pop('Letter Grade'))
    return X_train, y_train

In [9]:
def define_structure(X, Y):
    input_unit = X.shape[0] # size of input layer
    hidden_unit = X.shape[0] # hidden layer of size 4
    output_unit = Y.shape[0] # size of output layer
    return (input_unit, hidden_unit, output_unit)

In [10]:
def parameters_initialization(input_unit, hidden_unit, output_unit):
    np.random.seed(41)
    W1 = np.random.randn(hidden_unit, input_unit) * 0.01
    b1 = np.zeros((hidden_unit, 1))
    W2 = np.random.randn(output_unit, hidden_unit) * 0.01
    b2 = np.zeros((output_unit, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [11]:
def softmax(x):
    return scipy_softmax(x)

In [12]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    
    return A2, cache

In [13]:
def entropy_cost(A2, Y, parameters):
    #number of training example
    m = Y.shape[1]
    logprobs = np.multiply(np.log(A2), Y)
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
                                    
    return cost

In [14]:
def backward_propagation(parameters, cache, X, Y):
    #number of training example
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads

In [15]:
def gradient_descent(parameters, grads, learning_rate = 0.01):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
   
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1, "b1": b1,"W2": W2,"b2": b2}
    
    return parameters

In [16]:
def neural_network_model(X, Y, hidden_unit, num_iterations = 1000):
    np.random.seed(3)
    input_unit = define_structure(X, Y)[0]
    output_unit = define_structure(X, Y)[2]
    
    parameters = parameters_initialization(input_unit, hidden_unit, output_unit)
   
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = entropy_cost(A2, Y, parameters)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = gradient_descent(parameters, grads)
        if i % 5 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    return parameters

In [17]:
def prediction(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    return predictions

In [18]:
def get_error_score(df, columns):
    error_scores = {}
    sorted_semesters = sorted(set(df.iloc[:, 7]))
    for sem_idx in range(1, len(sorted_semesters)):
        training_sem = sorted_semesters[:sem_idx]
        test_sem = sorted_semesters[sem_idx]
        X_train, y_train = get_train_data(df, training_sem, columns)
        X_test = df[df.iloc[:, 7] == test_sem]
        X_test.drop('Semester', axis=1, inplace=True)
        y_test = le.transform(X_test.pop('Letter Grade'))
        
        X_train, X_test = standardize(X_train, X_test)
        
        X_train = X_train.T.to_numpy()   # (number of attributes, number of samples)
        y_train = y_train.reshape(1, y_train.shape[0])   # (1, number of samples)
        
        X_test = X_test.T.to_numpy()   # (number of attributes, number of samples)
        y_test = y_test.reshape(1, y_test.shape[0])   # (1, number of samples)
        
        parameters = neural_network_model(X_train, y_train, 58, num_iterations=5000)
        
        y_pred_test = prediction(parameters, X_test)
        rmse_test = round(np.sqrt(mean_squared_error(y_test[0], y_pred_test[0])), 3)
        mae_test = round(mean_absolute_error(y_test[0], y_pred_test[0]), 3)
        
        y_pred_train = prediction(parameters, X_train)
        rmse_train = round(np.sqrt(mean_squared_error(y_train[0], y_pred_train[0])), 3)
        mae_train = round(mean_absolute_error(y_train[0], y_pred_train[0]),3)
        
        error_scores.setdefault(sem_idx, {})
        error_scores[sem_idx]['rmse'] = [rmse_train, rmse_test]
        error_scores[sem_idx]['mae'] = [mae_train, mae_test]        
        
    return error_scores

In [19]:
model_results = {}

In [20]:
scores = get_error_score(df, columns)
model_results['NN'] = scores

/home/ali/.local/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Cost after iteration 0: 33.744129
Cost after iteration 5: 33.719827
Cost after iteration 10: 33.683103
Cost after iteration 15: 33.616869
Cost after iteration 20: 33.495200
Cost after iteration 25: 33.290022
Cost after iteration 30: 33.025667
Cost after iteration 35: 32.967791
Cost after iteration 40: 33.933481
Cost after iteration 45: 37.022637
Cost after iteration 50: 42.432604
Cost after iteration 55: 49.455903
Cost after iteration 60: 57.369132
Cost after iteration 65: 65.641812
Cost after iteration 70: 73.849694
Cost after iteration 75: 81.599883
Cost after iteration 80: 88.433319
Cost after iteration 85: 93.736312
Cost after iteration 90: 96.833655
Cost after iteration 95: 97.329526
Cost after iteration 100: 95.438081
Cost after iteration 105: 91.898392
Cost after iteration 110: 87.443037
Cost after iteration 115: 82.613813
Cost after iteration 120: 77.870224
Cost after iteration 125: 73.508232
Cost after iteration 130: 69.597373
Cost after iteration 135: 66.123990
Cost after ite

Cost after iteration 1140: 32.795791
Cost after iteration 1145: 32.797440
Cost after iteration 1150: 32.799095
Cost after iteration 1155: 32.800757
Cost after iteration 1160: 32.802425
Cost after iteration 1165: 32.804098
Cost after iteration 1170: 32.805777
Cost after iteration 1175: 32.807461
Cost after iteration 1180: 32.809149
Cost after iteration 1185: 32.810841
Cost after iteration 1190: 32.812538
Cost after iteration 1195: 32.814238
Cost after iteration 1200: 32.815942
Cost after iteration 1205: 32.817650
Cost after iteration 1210: 32.819360
Cost after iteration 1215: 32.821073
Cost after iteration 1220: 32.822789
Cost after iteration 1225: 32.824507
Cost after iteration 1230: 32.826227
Cost after iteration 1235: 32.827949
Cost after iteration 1240: 32.829673
Cost after iteration 1245: 32.831398
Cost after iteration 1250: 32.833125
Cost after iteration 1255: 32.834852
Cost after iteration 1260: 32.836581
Cost after iteration 1265: 32.838310
Cost after iteration 1270: 32.840040
C

Cost after iteration 2255: 33.119393
Cost after iteration 2260: 33.120443
Cost after iteration 2265: 33.121490
Cost after iteration 2270: 33.122534
Cost after iteration 2275: 33.123575
Cost after iteration 2280: 33.124613
Cost after iteration 2285: 33.125647
Cost after iteration 2290: 33.126679
Cost after iteration 2295: 33.127707
Cost after iteration 2300: 33.128732
Cost after iteration 2305: 33.129754
Cost after iteration 2310: 33.130773
Cost after iteration 2315: 33.131789
Cost after iteration 2320: 33.132802
Cost after iteration 2325: 33.133812
Cost after iteration 2330: 33.134819
Cost after iteration 2335: 33.135823
Cost after iteration 2340: 33.136823
Cost after iteration 2345: 33.137821
Cost after iteration 2350: 33.138816
Cost after iteration 2355: 33.139808
Cost after iteration 2360: 33.140797
Cost after iteration 2365: 33.141782
Cost after iteration 2370: 33.142765
Cost after iteration 2375: 33.143745
Cost after iteration 2380: 33.144722
Cost after iteration 2385: 33.145697
C

Cost after iteration 3365: 33.292268
Cost after iteration 3370: 33.292839
Cost after iteration 3375: 33.293408
Cost after iteration 3380: 33.293976
Cost after iteration 3385: 33.294543
Cost after iteration 3390: 33.295109
Cost after iteration 3395: 33.295673
Cost after iteration 3400: 33.296235
Cost after iteration 3405: 33.296797
Cost after iteration 3410: 33.297357
Cost after iteration 3415: 33.297915
Cost after iteration 3420: 33.298473
Cost after iteration 3425: 33.299029
Cost after iteration 3430: 33.299583
Cost after iteration 3435: 33.300136
Cost after iteration 3440: 33.300688
Cost after iteration 3445: 33.301239
Cost after iteration 3450: 33.301788
Cost after iteration 3455: 33.302336
Cost after iteration 3460: 33.302883
Cost after iteration 3465: 33.303428
Cost after iteration 3470: 33.303972
Cost after iteration 3475: 33.304515
Cost after iteration 3480: 33.305057
Cost after iteration 3485: 33.305597
Cost after iteration 3490: 33.306136
Cost after iteration 3495: 33.306673
C

...

Cost after iteration 0: 53.565970
Cost after iteration 5: 53.543540
Cost after iteration 10: 53.509846
Cost after iteration 15: 53.448999
Cost after iteration 20: 53.335921
Cost after iteration 25: 53.140674
Cost after iteration 30: 52.879334
Cost after iteration 35: 52.818564
Cost after iteration 40: 53.897149
Cost after iteration 45: 57.548967
Cost after iteration 50: 64.060259
Cost after iteration 55: 72.431865
Cost after iteration 60: 81.629596
Cost after iteration 65: 90.891934
Cost after iteration 70: 99.697196
Cost after iteration 75: 107.723434
Cost after iteration 80: 114.795060
Cost after iteration 85: 120.822353
Cost after iteration 90: 125.722677
Cost after iteration 95: 129.300773
Cost after iteration 100: 131.062853
Cost after iteration 105: 130.099746
Cost after iteration 110: 125.869693
Cost after iteration 115: 119.002292
Cost after iteration 120: 111.304192
Cost after iteration 125: 103.600226
Cost after iteration 130: 96.986485
Cost after iteration 135: 90.964114
Cos

Cost after iteration 1140: 52.674653
Cost after iteration 1145: 52.676413
Cost after iteration 1150: 52.678176
Cost after iteration 1155: 52.679940
Cost after iteration 1160: 52.681706
Cost after iteration 1165: 52.683473
Cost after iteration 1170: 52.685241
Cost after iteration 1175: 52.687010
Cost after iteration 1180: 52.688780
Cost after iteration 1185: 52.690551
Cost after iteration 1190: 52.692322
Cost after iteration 1195: 52.694093
Cost after iteration 1200: 52.695864
Cost after iteration 1205: 52.697635
Cost after iteration 1210: 52.699406
Cost after iteration 1215: 52.701176
Cost after iteration 1220: 52.702946
Cost after iteration 1225: 52.704714
Cost after iteration 1230: 52.706482
Cost after iteration 1235: 52.708249
Cost after iteration 1240: 52.710014
Cost after iteration 1245: 52.711778
Cost after iteration 1250: 52.713540
Cost after iteration 1255: 52.715301
Cost after iteration 1260: 52.717060
Cost after iteration 1265: 52.718818
Cost after iteration 1270: 52.720573
C

Cost after iteration 2250: 52.988027
Cost after iteration 2255: 52.989012
Cost after iteration 2260: 52.989994
Cost after iteration 2265: 52.990973
Cost after iteration 2270: 52.991949
Cost after iteration 2275: 52.992922
Cost after iteration 2280: 52.993892
Cost after iteration 2285: 52.994859
Cost after iteration 2290: 52.995823
Cost after iteration 2295: 52.996784
Cost after iteration 2300: 52.997741
Cost after iteration 2305: 52.998696
Cost after iteration 2310: 52.999648
Cost after iteration 2315: 53.000597
Cost after iteration 2320: 53.001543
Cost after iteration 2325: 53.002486
Cost after iteration 2330: 53.003426
Cost after iteration 2335: 53.004363
Cost after iteration 2340: 53.005298
Cost after iteration 2345: 53.006229
Cost after iteration 2350: 53.007158
Cost after iteration 2355: 53.008083
Cost after iteration 2360: 53.009006
Cost after iteration 2365: 53.009926
Cost after iteration 2370: 53.010843
Cost after iteration 2375: 53.011758
Cost after iteration 2380: 53.012669
C

Cost after iteration 3360: 53.149074
Cost after iteration 3365: 53.149603
Cost after iteration 3370: 53.150131
Cost after iteration 3375: 53.150657
Cost after iteration 3380: 53.151182
Cost after iteration 3385: 53.151706
Cost after iteration 3390: 53.152229
Cost after iteration 3395: 53.152750
Cost after iteration 3400: 53.153270
Cost after iteration 3405: 53.153789
Cost after iteration 3410: 53.154307
Cost after iteration 3415: 53.154823
Cost after iteration 3420: 53.155338
Cost after iteration 3425: 53.155852
Cost after iteration 3430: 53.156364
Cost after iteration 3435: 53.156876
Cost after iteration 3440: 53.157386
Cost after iteration 3445: 53.157894
Cost after iteration 3450: 53.158402
Cost after iteration 3455: 53.158908
Cost after iteration 3460: 53.159413
Cost after iteration 3465: 53.159917
Cost after iteration 3470: 53.160420
Cost after iteration 3475: 53.160921
Cost after iteration 3480: 53.161422
Cost after iteration 3485: 53.161921
Cost after iteration 3490: 53.162419
C

Cost after iteration 4470: 53.241262
Cost after iteration 4475: 53.241587
Cost after iteration 4480: 53.241911
Cost after iteration 4485: 53.242234
Cost after iteration 4490: 53.242557
Cost after iteration 4495: 53.242880
Cost after iteration 4500: 53.243201
Cost after iteration 4505: 53.243522
Cost after iteration 4510: 53.243843
Cost after iteration 4515: 53.244162
Cost after iteration 4520: 53.244481
Cost after iteration 4525: 53.244800
Cost after iteration 4530: 53.245118
Cost after iteration 4535: 53.245435
Cost after iteration 4540: 53.245752
Cost after iteration 4545: 53.246068
Cost after iteration 4550: 53.246383
Cost after iteration 4555: 53.246698
Cost after iteration 4560: 53.247012
Cost after iteration 4565: 53.247326
Cost after iteration 4570: 53.247639
Cost after iteration 4575: 53.247951
Cost after iteration 4580: 53.248263
Cost after iteration 4585: 53.248574
Cost after iteration 4590: 53.248885
Cost after iteration 4595: 53.249195
Cost after iteration 4600: 53.249504
C

In [21]:
with open('nn_results.json', 'w') as fw:
    json.dump(model_results, fw)